<a href="https://colab.research.google.com/github/Kwasi-Dankwa/medicaid-etl/blob/main/medicaid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extraction**



In [17]:
import requests
import pandas as pd

# Extraction
api_url = 'https://data.cms.gov/data-api/v1/dataset/be64fce3-e835-4589-b46b-024198e524a6/data'
response = requests.get(api_url)
data = response.json()




Transformation

In [18]:
# Transformation (converting JSON to DataFrame)
df = pd.DataFrame(data)

# Cleaning data
df.shape
df.info()
df.head()
df.describe(include='all')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Brnd_Name                        1000 non-null   object
 1   Gnrc_Name                        1000 non-null   object
 2   Tot_Mftr                         1000 non-null   object
 3   Mftr_Name                        1000 non-null   object
 4   Tot_Spndng_2018                  1000 non-null   object
 5   Tot_Dsg_Unts_2018                1000 non-null   object
 6   Tot_Clms_2018                    1000 non-null   object
 7   Avg_Spnd_Per_Dsg_Unt_Wghtd_2018  1000 non-null   object
 8   Avg_Spnd_Per_Clm_2018            1000 non-null   object
 9   Outlier_Flag_2018                1000 non-null   object
 10  Tot_Spndng_2019                  1000 non-null   object
 11  Tot_Dsg_Unts_2019                1000 non-null   object
 12  Tot_Clms_2019                    10

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2018,Tot_Dsg_Unts_2018,Tot_Clms_2018,Avg_Spnd_Per_Dsg_Unt_Wghtd_2018,Avg_Spnd_Per_Clm_2018,Outlier_Flag_2018,...,Avg_Spnd_Per_Clm_2021,Outlier_Flag_2021,Tot_Spndng_2022,Tot_Dsg_Unts_2022,Tot_Clms_2022,Avg_Spnd_Per_Dsg_Unt_Wghtd_2022,Avg_Spnd_Per_Clm_2022,Outlier_Flag_2022,Chg_Avg_Spnd_Per_Dsg_Unt_21_22,CAGR_Avg_Spnd_Per_Dsg_Unt_18_22
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,262,219,20,250,581,578,549,581,581,3,...,751,3,807,800,688,807,807,2,840,762
top,Acyclovir*,Aripiprazole,1,Overall,,,,,,0,...,,0,2632099.12,690,17,11.212483995,334.83006233,0,,0
freq,34,34,925,269,264,264,264,264,264,491,...,61,601,2,3,13,2,2,597,61,52


In [19]:
# Standardizing column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [20]:
#fixing datatypes
# Ensure spending columns are numeric
spending_cols = [col for col in df.columns if 'spndng' in col]
df[spending_cols] = df[spending_cols].apply(pd.to_numeric, errors='coerce')


# Ensure avg columns are numeric
avg_cols = [col for col in df.columns if 'avg' in col]
df[avg_cols] = df[avg_cols].apply(pd.to_numeric, errors='coerce')

# Ensure tot columns are numeric
tot_cols = [col for col in df.columns if 'tot' in col]
df[tot_cols] = df[tot_cols].apply(pd.to_numeric, errors='coerce')



In [21]:
# Removing irrelevant columns
df = df.drop(columns=[col for col in df.columns if 'outlier' in col])
columns_to_drop = [
    'tot_mftr',
    'tot_dsg_unts_2018', 'tot_dsg_unts_2019', 'tot_dsg_unts_2020', 'tot_dsg_unts_2021', 'tot_dsg_unts_2022',
    'tot_clms_2018', 'tot_clms_2019', 'tot_clms_2020', 'tot_clms_2021', 'tot_clms_2022',
    'avg_spnd_per_clm_2018', 'avg_spnd_per_clm_2019', 'avg_spnd_per_clm_2020', 'avg_spnd_per_clm_2021', 'avg_spnd_per_clm_2022'
]

df = df.drop(columns=columns_to_drop) # dropping columns



In [22]:
# Replacing null values with 0 in spnding columns
# replacing null values in spending
spending_cols_to_impute = [col for col in df.columns if 'spndng' in col]
for col in spending_cols_to_impute:
  df[col] = df[col].fillna(0)

# replacing null values in wghtd columns with 0
weighted_cols_to_impute = [col for col in df.columns if 'wghtd' in col]
for col in weighted_cols_to_impute:
  df[col] = df[col].fillna(0)

# replace null values in avg columns with 0
average_cols_to_impute = [col for col in df.columns if 'avg' in col]
for col in average_cols_to_impute:
  df[col] = df[col].fillna(0)

df.isnull().sum() # this verifies the relevant data has been cleaned

,0
brnd_name,0
gnrc_name,0
mftr_name,0
tot_spndng_2018,0
avg_spnd_per_dsg_unt_wghtd_2018,0
tot_spndng_2019,0
avg_spnd_per_dsg_unt_wghtd_2019,0
tot_spndng_2020,0
avg_spnd_per_dsg_unt_wghtd_2020,0
tot_spndng_2021,0


Load Phase

In [23]:
# Save cleaned dataset as csv
df.to_csv('medicaid_drug_spending_cleanv2.csv', index=False)


In [24]:
# load transformed data to dbbrowser

import sqlite3

# Load Phase - Save to SQLite database
# Creating new SQLite database
conn = sqlite3.connect('medicaid_drug_spendingv2.db')

# Writing the DataFrame to a SQLite table
df.to_sql('drug_spending', conn, if_exists='replace', index=False)

# Closing the database connection
conn.close()

print("Data successfully loaded into medicaid_drug_spending.db")


Data successfully loaded into medicaid_drug_spending.db
